In [7]:
# -*- coding: utf-8 -*-
"""
Created on Fri Dec 20 02:16:07 2019

@author: tommy
"""

#%% import libs
import pandas as pd
import numpy as np

#%% edhec funcs

def discount(t, r):
    """
    Compute the price of a pure discount bond that pays a dollar at time period t
    and r is the per-period interest rate
    returns a |t| x |r| Series or DataFrame
    r can be a float, Series or DataFrame
    returns a DataFrame indexed by t
    """
    discounts = pd.DataFrame([(r+1)**-i for i in t])
    discounts.index = t
    return discounts

def pv(flows, r):
    """
    Compute the present value of a sequence of cash flows given by the time (as an index) and amounts
    r can be a scalar, or a Series or DataFrame with the number of rows matching the num of rows in flows
    """
    dates = flows.index
    discounts = discount(dates, r)
    return discounts.multiply(flows, axis='rows').sum()

def bond_cash_flows(maturity, principal=100, coupon_rate=0.03, coupons_per_year=12):
    """
    Returns the series of cash flows generated by a bond,
    indexed by the payment/coupon number
    """
    n_coupons = round(maturity*coupons_per_year)
    coupon_amt = principal*coupon_rate/coupons_per_year
    coupons = np.repeat(coupon_amt, n_coupons)
    coupon_times = np.arange(1, n_coupons+1)
    cash_flows = pd.Series(data=coupon_amt, index=coupon_times)
    cash_flows.iloc[-1] += principal
    return cash_flows
    

def bond_price(maturity, principal=100, coupon_rate=0.03, coupons_per_year=12, discount_rate=0.03):
    """
    Computes the price of a bond that pays regular coupons until maturity
    at which time the principal and the final coupon is returned
    This is not designed to be efficient, rather,
    it is to illustrate the underlying principle behind bond pricing!
    If discount_rate is a DataFrame, then this is assumed to be the rate on each coupon date
    and the bond value is computed over time.
    i.e. The index of the discount_rate DataFrame is assumed to be the coupon number
    """
    if isinstance(discount_rate, pd.DataFrame):
        pricing_dates = discount_rate.index
        prices = pd.DataFrame(index=pricing_dates, columns=discount_rate.columns)
        for t in pricing_dates:
            prices.loc[t] = bond_price(maturity-t/coupons_per_year, principal, coupon_rate, coupons_per_year,
                                      discount_rate.loc[t])
        return prices
    else: # base case ... single time period
        if maturity <= 0: return principal+principal*coupon_rate/coupons_per_year
        cash_flows = bond_cash_flows(maturity, principal, coupon_rate, coupons_per_year)
        return pv(cash_flows, discount_rate/coupons_per_year) 
    
def macaulay_duration(flows, discount_rate):
    """
    Computes the Macaulay Duration of a sequence of cash flows, given a per-period discount rate
    """
    discounted_flows = discount(flows.index, discount_rate)*pd.DataFrame(flows)
    weights = discounted_flows/discounted_flows.sum()
    return np.average(flows.index, weights=weights.iloc[:,0])

def match_durations(cf_t, cf_s, cf_l, discount_rate):
    """
    Returns the weight W in cf_s that, along with (1-W) in cf_l will have an effective
    duration that matches cf_t
    """
    d_t = macaulay_duration(cf_t, discount_rate)
    d_s = macaulay_duration(cf_s, discount_rate)
    d_l = macaulay_duration(cf_l, discount_rate)
    return (d_l - d_t)/(d_l - d_s)

#%%























In [8]:
b1 = bond_price(15, 1000, .05, 2,.05 )
b1

0    1000.0
dtype: float64

In [9]:
b2 = bond_price(5, 1000, .06, 4,.05 )
b2

0    1043.99829
dtype: float64

In [10]:
b3 = bond_price(10, 1000, .0, 1,.05 )
b3

0    613.913254
dtype: float64

In [21]:
d_l = macaulay_duration(bond_cash_flows(15, 1000, .05, 2),  .05/2) /2
d_l

10.72677495379012

In [22]:
d_s = macaulay_duration(bond_cash_flows(5, 1000, .06, 4 ),  .05/4) / 4
d_s

4.373363222636413

In [13]:
macaulay_duration(bond_cash_flows(10, 1000, .0, 1 ),  .05)

10.0

In [14]:
liabilities = pd.Series(data = [100000, 200000, 300000], index=[3,5,10])
liabilities

3     100000
5     200000
10    300000
dtype: int64

In [15]:
d_t = macaulay_duration(liabilities, .05)
d_t

6.750917852744651

In [23]:
w_s = (d_l - d_t)/(d_l - d_s)

In [24]:
w_s

0.6257830075060314

In [28]:
# 10
d_s = macaulay_duration(bond_cash_flows(5, 1000, .06, 4 ),  .05/4) / 4
d_s

d_l = macaulay_duration(bond_cash_flows(10, 1000, 0, 1 ),  .05)
d_l

10.0

In [29]:
w_s = (d_l - d_t)/(d_l - d_s)
w_s

0.5774465770256697

## Copy of Questions

1.Question 1
In the following questions, we will be working with three bonds:

B1 is a 15 Year Bond with a Face Value of $1000$ that pays a 5% coupon semi-annually (2 times a year)
B2 is a 5 Year Bond with a Face value of $1000$ that pays a $6\%$ coupon quarterly (4 times a year)
B3 is a 10 Year Zero-Coupon Bond with a Face Value of $1000 (Hint: you can still use the erk.bond_cash_flows() and erk.bond_price() by setting the coupon amount to 0% and coupons_per_year to 1) Assume the yield curve is flat at 5%. Duration refers to Macaulay Duration
Hint: the macaulay_duration function gives as output the duration expressed in periods and not in years. If you want to get the yearly duration you need to divide the duration for coupons_per_year;

e.g.: duarion_B2 = erk.macaulay_duration(flows_B2, 0.05/4)/4

Which of the three bonds is the most expensive?
1 / 1 point


B2


B1


B3

Correct
2.Question 2
Which of the three bonds is the least expensive?
1 / 1 point


B3


B1


B2

Correct
3.Question 3
What is the price of the 10 Year Zero Coupon Bond B3?

Enter the answer rounded to the nearest Dollar. e.g. if you compute the price as $45.35, enter the number 45.
1 / 1 point
614

Correct
4.Question 4
Which of the three bonds has the highest (Macaulay) Duration?
1 / 1 point


B2


B1


B3

Correct
5.Question 5
Which of the three bonds has the lowest (Macaulay) Duration?
1 / 1 point


B2


B3


B1

Correct
6.Question 6
What is the duration of the 5 year bond B2?

Enter the answer as a number - e.g. for 5.43 years, enter 5.43.
1 / 1 point
4.37

Correct
7.Question 7
Assume a sequence of 3 liabilities of $100,000, $200,000 and $300,000 that are 3, 5 and 10 years away, respectively. What is the Duration of the liabilities?

(Reminder: Assume the yield curve is flat at 5%. Duration refers to Macaulay Duration)

Enter the answer as a number - e.g. for 5.43 years, enter 5.43.
1 / 1 point
6.75

Correct
8.Question 8
Assuming the same set of liabilities as the previous question (i.e. a sequence of 3 liabilities of $100,000, $200,000 and $300,000 that are 3, 5 and 10 years away, respectively) build a Duration Matched Portfolio of B1 and B2 to match these liabilities. What is the weight of B2 in the portfolio? (Hint: the code we developed in class erk.match_durations() assumes that all the bonds have the same number of coupons per year. This is not the case here, so you will either need to enhance the code or compute the weight directly e.g. by entering the steps in a Jupyter Notebook Cell or at the Python Command Line)

Enter the weight as a percentage. For instance, if you compute the weight as 34.66%, enter 34.66.
1 / 1 point
62.58

Correct
9.Question 9
Assume you can use any of the bonds B1, B2 and B3 to build a duration matched bond portfolio matched to the liabilities. Which combination of 2 bonds can you NOT use to build a duration matched bond portfolio?
1 / 1 point


B1+B3


B2+B3


B1+B2


ANY PAIR WILL WORK


10.Question 10
Assuming the same liabilities as the previous questions (i.e. a sequence of 3 liabilities of $100,000, $200,000 and $300,000 that are 3, 5 and 10 years away, respectively), build a Duration Matched Portfolio of B2 and B3 to match the liabilities.

What is the weight of B2 in this portfolio?

Enter the answer as a single number. For instance, if you compute the weight as 65.32% enter 65.32.